## 반드시 알아야 하는 파이토치 스킬
#### 텐서
Scalar: 우리가 흔히 알고 있는 상수 값. 하나의 값을 표현할 때 1개의 수치로 표현

Vector: 하나의 값을 표현할 때 2개 이상의 수치로 표현

Matrix: 2개 이상의 벡터 값을 통합해 구성된 값. 선형 대수의 기본 단위. 2차원의 배열

Tensor: 2차원 이상의 배열

In [1]:
#텐서 값 간의 사칙연산 예제
import torch
tensor1 = torch.tensor([[[1.,2.], [3., 4.]], [[5., 6.], [7., 8,]]])
print(tensor1)

tensor2 = torch.tensor([[[9.,10.], [11., 12.]], [[13., 14.], [15., 16,]]])
print(tensor2)

tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]])
tensor([[[ 9., 10.],
         [11., 12.]],

        [[13., 14.],
         [15., 16.]]])


In [2]:
sum_tensor = tensor1 +tensor2
print(sum_tensor)

tensor([[[10., 12.],
         [14., 16.]],

        [[18., 20.],
         [22., 24.]]])


In [3]:
sub_tensor = tensor1 -tensor2
print(sub_tensor)

mul_tensor = tensor1 *tensor2
print(mul_tensor)

div_tensor = tensor1 /tensor2
print(div_tensor)

tensor([[[-8., -8.],
         [-8., -8.]],

        [[-8., -8.],
         [-8., -8.]]])
tensor([[[  9.,  20.],
         [ 33.,  48.]],

        [[ 65.,  84.],
         [105., 128.]]])
tensor([[[0.1111, 0.2000],
         [0.2727, 0.3333]],

        [[0.3846, 0.4286],
         [0.4667, 0.5000]]])


torch 모듈에 내장된 메서드로 사칙연산 + 내적연산

In [4]:
torch.add(tensor1, tensor2)

tensor([[[10., 12.],
         [14., 16.]],

        [[18., 20.],
         [22., 24.]]])

In [5]:
torch.sub(tensor1, tensor2)

tensor([[[-8., -8.],
         [-8., -8.]],

        [[-8., -8.],
         [-8., -8.]]])

In [6]:
torch.mul(tensor1, tensor2)

tensor([[[  9.,  20.],
         [ 33.,  48.]],

        [[ 65.,  84.],
         [105., 128.]]])

In [7]:
torch.div(tensor1, tensor2)

tensor([[[0.1111, 0.2000],
         [0.2727, 0.3333]],

        [[0.3846, 0.4286],
         [0.4667, 0.5000]]])

In [8]:
#텐서 내적연산
torch.matmul(tensor1, tensor2)

tensor([[[ 31.,  34.],
         [ 71.,  78.]],

        [[155., 166.],
         [211., 226.]]])

## Autograd & 관련 배경
torch.autograd: 신경망 학습을 지원하는 PyTorch의 자동 미분엔진

### 배경
신경망(NN:Neural Network) 어떤 입력 데이터에 대해 실행되는 중첩(nested)된 함수들의 모음. 이 함수들은 PyTorch에서 Tensor로 저장되는, 가중치(weight)와 편향(bias)로 구성된 매개변수들로 정의됨

#### 신경망 학습 방법
순전파(Forward Propagation): 순전파 단계에서, 신경망은 정답을 맞추기 위해 최선의 추측(best guess)을 함. 추측을 위해 입력 데이터를 각 함수들에서 실행

역전파(Backward Propagation): 역전파 단계에서, 신경망은 추측한 값에서 발생한 오류(error)에 비례하여(proportionate) 매개변수들을 적절히 조절함. 출력(output)로부터 역방향으로 이동하면서 오류에 대한 함수들의 매개변수들의 미분값(gradient)을 수집하고, 경사하강법(gradient descent)을 사용하여 매개변수들을 최적화 함

이번 예제는 간단한 딥러닝 모델을 설계하고 방정식 내에 존재하는 파라미터를 업데이트

In [9]:
import torch

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

BATCH_SIZE는 딥러닝 모델에서 파라미터를 업데이트할 때 계산되는 데이터 수

INPUT_SIZE는 딥러닝 모델에서의 Input의 크기이자 입력층의 노드 수

HIDDEN_SIZE는 딥러닝 모델에서 Input을 다수의 파라미터를 이용해 계산한 결과에 한 번 더 계산되는 파라미터 수

OUTPUT_SIZE는 딥러닝 모델에서 최종으로 출력되는 값의 벡터의 크기

In [10]:
BATCH_SIZE = 64
INPUT_SIZE = 1000
HIDDEN_SIZE = 100
OUTPUT_SIZE = 10

In [11]:
#randn 매서드를 이용해 데이터와 파라미터를 설정
#randn은 평균이 0, 표준편차가 1인 정규분포에서 샘플링한 값으로, 데이터를 만든다는 것을 의미하며, 데이터를 만들어 낼 때 데이터 모양 설정 가능
x = torch.randn(BATCH_SIZE, INPUT_SIZE, device = DEVICE, dtype = torch.float, requires_grad = False)
y= torch.randn(BATCH_SIZE, OUTPUT_SIZE, device = DEVICE, dtype = torch.float, requires_grad = False)
#업데이트할 파라미터 값을 설정
w1 = torch.randn(INPUT_SIZE, HIDDEN_SIZE, device = DEVICE, dtype = torch.float, requires_grad = True)
w2 = torch.randn(HIDDEN_SIZE, OUTPUT_SIZE, device = DEVICE, dtype = torch.float, requires_grad = True)

In [18]:
#learning_rate 설정에 따라 Gradient 값에 따른 학습 정도가 결정
learning_rate = 1e-6

for t in range(1, 501):
    y_pred = x.mm(w1).clamp(min = 0).mm(w2) 
    # Input x와 Parameter w1간의 행렬 곱을 이용해 나온 결과 값 계산
    # clamp 매서드 활용해 비선형 함수를 적용
    loss = (y_pred -y).pow(2).sum()
    if t%100 == 0:
        print("Iteration: ", t, "\t", "Loss: ", loss.item())
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

Iteration:  100 	 Loss:  1.5029465430416167e-05
Iteration:  200 	 Loss:  9.635405149310827e-06
Iteration:  300 	 Loss:  6.993234819674399e-06
Iteration:  400 	 Loss:  5.416171461547492e-06
Iteration:  500 	 Loss:  4.43070075561991e-06


500번의 반복문을 실행하면서 Loss 값이 줄어드는 것을 확인할 수 있음. Loss 값이 줄어든 다는 것은 Input이 w1과 w2를 통해 계산된 결과값과 y 값이 점점 비슷해진다는 것을 의미하며 y값과 비슷한 Output을 계산할 수 있도록 w1과 w2 계산된다는 것을 알 수 있음